In [6]:
import os
import datetime
import numpy as np 
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
from scipy.stats import binned_statistic
import CRTS_paper_modules as mod
reload(mod)

<module 'CRTS_paper_modules' from 'CRTS_paper_modules.pyc'>

Introduce all necessary functions:
    - reading the SDSS-CRTS cross-matched catalogue 
    - make cuts in the available objecs according to selection criteria 

In [7]:
cols1, qso_cat = mod.get_qso_catalog(catalog='DB_QSO') 
cols2 , star_cat= mod.get_stars_catalog()



Zipping CRTS-SDSS quasars catalog from  CRTS_SDSS_cross_matched_qso_DB_QSO_catalog.txt  ...
Read in  7601 , quasars from CRTS
zipping CRTS-SDSS stars catalog...
Read in catalog for  48250 , stars from CRTS


Introduce xi, ei reading functions

## 1) Use SDSS r-mag  only 

In [21]:
Min = 17
Max = 18
magnitudes = ['r']

# Where to save the samples of log(tau), as well as the table results  ... 
outDir = os.path.join(os.getcwd(), 'Fig_3_data', datetime.datetime.now().strftime('%Y-%m-%d')+ '_histogram_table/')

# Create the dir if it does not yet exist ... 
if not os.path.exists(outDir): os.system('mkdir %s' % outDir)
print('Today we will save our Fig_3 files in %s'%outDir)

objects_in_cut = {}

for mag in magnitudes : 
    cut_mag = mag
    report_mag = mag
    
    print('\nUsing now only lightcurves with SDSS  %f< %s < %f' % (Min, cut_mag, Max))
    print('\n Reporting SDSS %s  '% report_mag)

    good_ids_S_blue = mod.cut_stars(star_cat=star_cat ,mMin = Min, mMax=Max, mErrMax = 0.3, gi_Min = -1,
                                              gi_Max=1, cut_mag=cut_mag + '_mMed')
    
    good_ids_S_red = mod.cut_stars(star_cat=star_cat, mMin = Min, mMax=Max, mErrMax = 0.3, gi_Min = 1, 
                                           gi_Max=3, cut_mag=cut_mag + '_mMed')
    
    good_ids_QSO = mod.cut_qso(qso_cat=qso_cat,mMin = Min, mMax=Max, mErrMax = 0.3, 
                                               cut_mag=cut_mag)
    objects_in_cut[mag] = {'starsB':good_ids_S_blue, 'starsR':good_ids_S_red, 
                           'qso':good_ids_QSO}
    

Today we will save our Fig_3 files in /astro/users/suberlak/Desktop/SDSS/SF_plotting/Fig_3_data/2016-02-12_histogram_table/

Using now only lightcurves with SDSS  17.000000< r < 18.000000

 Reporting SDSS r  

 These cuts reduced the number of stars  in the sample from 48250  to  2781

 These cuts reduced the number of stars  in the sample from 48250  to  2982

 These cuts reduced the number of qso  in the sample from 7601  to  185


## 3) Making r_cut is the same as what I read in ... (since I don't need g_cut) 

In [22]:
bins = {}
bin_types = ['r_cut']

objects = objects_in_cut['r'].keys()

# first need to explicitly initialize the dictionaries 
for b in bin_types:
    bins[b] = {}
    
for obj in objects : 
    bins['r_cut'][obj] =  objects_in_cut['r'][obj]

    


## 4) Read in all xi, ei for objects in bins

In [23]:
inDirStars   = 'sf_file_per_LC/star/'
inDirQSO = 'sf_file_per_LC/qso/'

out_dic = {}

#for b in bins.keys():
# read in only r_cut 

b = 'r_cut'
print 'Reading in xi, ei for bin ', b
out_dic[b] = {}   # initialize the dic 

good_ids_S_blue = bins[b]['starsB']
good_ids_S_red = bins[b]['starsR']
good_ids_QSO = bins[b]['qso']

qso, starB, starR = mod.read_xi_ei(inDirStars, good_ids_S_blue, good_ids_S_red, inDirQSO,
              good_ids_QSO)

# put into a dictionary : makes it more explicit 
out_dic[b] = {'starsB': starB, 'starsR': starR, 'qso':qso}


# Straight after reading-in xi, ei,   one can proceed directly to part 9) (one bin) or 10 : all bins sigma comparison 
# or to Saving just the log(tau) samples of xi, tau, ei. 


Reading in xi, ei for bin  r_cut
making new delflx, tau, xi arrays


----- Already read 100% of qso 

----- Already read 99% of Blue Stars 

----- Already read 99% of Red Stars returning xi, ei for ... 185 objects


## 10) Save to a file all points for samples of log(tau) 

In [24]:
## Save to a file points for log(tau) samples  for QSO, stars 

b = 'r_cut'   # or g_cut

tau_min_arr = [0,   2.3, 2.8, 3.2]
tau_max_arr = [1.7, 2.5, 3.0, 3.4]
print 'Saving the samples of xi, ei  for log(tau) ranges in ', outDir
for obj in ['qso','starsB']:
    #obj = 'qso'  # or starsB,  starsR 
    m_ij = out_dic[b][obj][0]
    tau =  out_dic[b][obj][1]
    e_ij =  out_dic[b][obj][2]
    
    for i in range(len(tau_min_arr)):
        m1 = tau_min_arr[i] < np.log10(tau)
        m2 = np.log10(tau) < tau_max_arr[i]
        mask =  m1 * m2 

        data = np.column_stack((m_ij[mask], tau[mask], e_ij[mask]))
        fname = b+'_'+str(Min)+'-'+str(Max)+'_'+obj+'_mi_tau_ei-log_tau_'+\
                str(tau_min_arr[i])+'-'+str(tau_max_arr[i])+'.txt'
        print 'Saved', fname
        np.savetxt(outDir+fname, data, fmt='%s', delimiter= ' ')


Saving the samples of xi, ei  for log(tau) ranges in  /astro/users/suberlak/Desktop/SDSS/SF_plotting/Fig_3_data/2016-02-12_histogram_table/
Saved r_cut_17-18_qso_mi_tau_ei-log_tau_0-1.7.txt
Saved r_cut_17-18_qso_mi_tau_ei-log_tau_2.3-2.5.txt
Saved r_cut_17-18_qso_mi_tau_ei-log_tau_2.8-3.0.txt
Saved r_cut_17-18_qso_mi_tau_ei-log_tau_3.2-3.4.txt
Saved r_cut_17-18_starsB_mi_tau_ei-log_tau_0-1.7.txt
Saved r_cut_17-18_starsB_mi_tau_ei-log_tau_2.3-2.5.txt
Saved r_cut_17-18_starsB_mi_tau_ei-log_tau_2.8-3.0.txt
Saved r_cut_17-18_starsB_mi_tau_ei-log_tau_3.2-3.4.txt


## 4) a) Grab the table for the log(tau) sample 

In [25]:
# Select   log(tau)  sample  from each set of xi,ei  

def make_table(out_dic=out_dic, qso_cat = qso_cat , star_cat = star_cat, tau_min=0, tau_max = 1.7, outDir=None):
    out_dic_cut = {}
    for b in out_dic.keys():
    #b = 'r_cut'    
        out_dic_cut[b] = {}

        for obj in out_dic[b].keys() : 

            print '\n Selecting sample log(tau) for bin', b, ' for ', obj

            xi  = out_dic[b][obj][0]
            tau = out_dic[b][obj][1]
            ei  = out_dic[b][obj][2]
            n   = out_dic[b][obj][3]

            # make log(tau) cut
            #print('Before log(tau) cut we have %d objects and %d delta_mag points'% (len(np.unique(n)),len(xi)))

            m1 = tau_min < np.log10(tau)
            m2 = np.log10(tau) < tau_max
            mask_tau =  m1 * m2 

            print('After log(tau) cut we have %d objects and %d delta_mag points'% (len(np.unique(n[mask_tau])),
                                                                                    len(xi[mask_tau]) ))

            # save xi,ei,tau, n  that satisfy the cut 
            out_dic_cut[b][obj] = [xi[mask_tau], tau[mask_tau], ei[mask_tau], n[mask_tau]]


    # Figure out what objects are in the log(tau) < 1.7 samples : make new lists of names, 
    # similar to making bins in the first place 

    # Now grab unique names of objects that appear in my samples
    new_bins = {}
    for b in out_dic_cut.keys():
        new_bins[b] = {}
        #print '\n b=', b
        for obj in out_dic_cut[b].keys():
            #print 'obj=', obj
            uniq_names = np.unique(out_dic_cut[b][obj][3]) 
            new_bins[b][obj] = uniq_names
            #print len(uniq_names)


    # Grab lightcurve - averaged properties , averaged over all objects 
    # of a given type in my sample 

    # Stays the same because exactly the same objects are in my log(tau) < 1.7 cut 

    new_bins_props = {}
    new_bins_arrays = {}

    for b in new_bins.keys():
        #print 'b=', b
        new_bins_props[b] = {}
        new_bins_arrays[b] = {}

        for obj in new_bins[b].keys():
            #print 'obj=', obj
            if obj == 'qso' : 
                CRTS_avg_err = [qso_cat['CRTS_avg_e'][qso_cat['CRTS_id']==qso][0] for qso
                               in new_bins[b][obj]]
                CRTS_avg_mag = [qso_cat['CRTS_avg_m'][qso_cat['CRTS_id']==qso][0] for qso
                               in new_bins[b][obj]]

            else : 

                CRTS_avg_err = [star_cat['CRTS_Merr'][star_cat['crts_id']== float(star)][0] for star
                               in new_bins[b][obj]]
                CRTS_avg_mag = [star_cat['CRTS_M'][star_cat['crts_id']==float(star)][0] for star
                               in new_bins[b][obj]]

            # change into floats and arrays 
            arr1 = np.array(CRTS_avg_err).astype(float)
            arr2 = np.array(CRTS_avg_mag).astype(float)
            new_bins_arrays[b][obj] = {'CRTS_avg_err' : arr1,
                                       'CRTS_avg_mag' : arr2}

            med_avg_err = np.median(arr1)
            med_avg_mag = np.median(arr2) 

            new_bins_props[b][obj] = {'median_CRTS_avg_err':med_avg_err, 
                                     'median_CRTS_avg_mag':med_avg_mag
                                    }

            # Grab magnitude difference quantities, corresponding to individual delta_mag,  
    # delta_mag_err, delta_tau  points in my sample 

    for b in new_bins.keys():
        print '\n', b
        for obj in new_bins[b].keys():

            # Grab  delta_magnitude and corresponding errors from lightcurve points 
            # added in quadrature 
            ei = out_dic_cut[b][obj][2]
            mi = out_dic_cut[b][obj][0]
            #print 'len(ei) = ', len(ei)
            # Calculate median  ei  for each bin
            new_bins_props[b][obj]['median_CRTS_ei'] =  np.median(ei)

            # Calculate robust standard deviation for delta mag 
            if len(mi) > 0 : 

                new_bins_props[b][obj]['stdev_robust_mi'] = 0.7414 *(np.percentile(mi,75) - np.percentile(mi,25))
            else : 
                new_bins_props[b][obj]['stdev_robust_mi'] = np.nan

            # Calculate robust standard deviation for chi_ij = (delta_mag_ij / error_ij)
            if len(mi) > 0 and len(ei) > 0 : 
                chi = mi / ei 
                new_bins_props[b][obj]['stdev_robust_chi'] = 0.7414 *(np.percentile(chi,75) - np.percentile(chi,25))
            else : 
                new_bins_props[b][obj]['stdev_robust_chi'] = np.nan

    # save to a file
    if not os.path.exists(outDir): os.system('mkdir %s' % outDir) 
    savefile = 'aft_CRTS_'+str(Min)+'-'+str(Max)+'_mag_log_t_'+str(tau_min)+'-'+str(tau_max)+'.txt'
    f = open(outDir+savefile,"w")

    # loop over all lines to display
    for prop in new_bins_props[b][obj].keys():

        data = [[],[],[]]

        i = 0
        for obj in new_bins_props[b].keys():
            data[i].append(obj)
            for b in new_bins_props.keys():
                data[i].append(new_bins_props[b][obj][prop])
            i += 1

        # assume that your data rows are tuples
        temp_head = "{0:8s}|{1:8}" # column widths: 8, 10, 15, 7, 10
        temp_lines = "{0:8s}|{1:8.3f}"



        s_title = '        ' +  prop
        print s_title
        f.write(s_title+'\n')

        s_sep = '    '
        print s_sep
        f.write(s_sep+'\n')

        s_head = temp_head.format(" ", "r_cut", "g_cut")
        print  s_head
        f.write(s_head+'\n')

        for rec in data: 
            s_line  = temp_lines.format(*rec)
            print s_line
            f.write(s_line+'\n')
        print '    '
        f.write('   '+'\n')

    # At the end, close the file 
    f.close()
    print 'Saved all to a file', outDir+savefile  
    print('For range of tau between %.1f  and %.1f, f_c_Blue_stars =  %.4f  (=stdev_robust_chi for blue stars)'%
    (tau_min, tau_max, new_bins_props['r_cut']['starsB']['stdev_robust_chi']))

In [26]:
tau_min_arr = [0,   2.3, 2.8, 3.2]
tau_max_arr = [1.7, 2.5, 3.0, 3.4]

for i in range(len(tau_min_arr)):
    make_table(tau_min = tau_min_arr[i], tau_max = tau_max_arr[i], outDir=outDir)


 Selecting sample log(tau) for bin r_cut  for  qso
After log(tau) cut we have 185 objects and 33371 delta_mag points

 Selecting sample log(tau) for bin r_cut  for  starsB
After log(tau) cut we have 2781 objects and 459841 delta_mag points

 Selecting sample log(tau) for bin r_cut  for  starsR
After log(tau) cut we have 2982 objects and 517736 delta_mag points

r_cut
        median_CRTS_ei
    
        |r_cut   
qso     |   0.071
starsB  |   0.069
starsR  |   0.066
    
        stdev_robust_mi
    
        |r_cut   
qso     |   0.072
starsB  |   0.060
starsR  |   0.055
    
        median_CRTS_avg_mag
    
        |r_cut   
qso     |  17.578
starsB  |  17.442
starsR  |  17.364
    
        stdev_robust_chi
    
        |r_cut   
qso     |   1.012
starsB  |   0.864
starsR  |   0.826
    
        median_CRTS_avg_err
    
        |r_cut   
qso     |   0.050
starsB  |   0.050
starsR  |   0.048
    
Saved all to a file /astro/users/suberlak/Desktop/SDSS/SF_plotting/Fig_3_data/2016-02-12_hi